# 4. Free-fermion models

For a given Hermitian matrix $h \in \mathbb{C}^{L \times L}$, we define the corresponding free-fermion Hamiltonian (containing solely quadratic creation and annihilation operator terms) as
\begin{equation*}
H = \sum_{j,k=0}^{L-1} h_{jk} a_j^{\dagger} a_k.
\end{equation*}
Note that the matrix representation of $H \in \mathbb{C}^{2^L \times 2^L}$ is high-dimensional (scales exponentially with $L$), so conventional linear algebra operations on $H$ become numerically infeasible for $L \gtrsim 20$. For the numerical experiments below, we use a small $L$ such that we can still easily construct $H$ as a (sparse) matrix.

Surprisingly, we can use an orbital basis rotation to bring $H$ into a diagonal form, unaffected by curse-of-dimensionality problems, as follows. First, diagonalize the coefficient matrix $h$:
\begin{equation*}
h = u \begin{pmatrix} \epsilon_0 & & & \\ & \epsilon_1 & & \\ & & \ddots & \\ & & & \epsilon_{L-1} \end{pmatrix} u^{\dagger},
\end{equation*}
where $u \in \mathbb{C}^{L \times L}$ is unitary and $\epsilon_0, \dots, \epsilon_{L-1}$ are the real eigenvalues of $h$. Inserting into the definition of $H$ leads to
\begin{equation*}
H = \sum_{j,k=0}^{L-1} \sum_{\ell=0}^{L-1} u_{j\ell} \epsilon_{\ell} u_{k\ell}^* a_j^{\dagger} a_k = \sum_{\ell=0}^{L-1} \epsilon_{\ell} \left(\sum_{j=0}^{L-1} u_{j\ell} a_j^{\dagger}\right) \left(\sum_{k=0}^{L-1} u_{k\ell}^* a_k\right) = \sum_{\ell=0}^{L-1} \epsilon_{\ell} \tilde{a}_{\ell}^{\dagger} \tilde{a}_{\ell} = \sum_{\ell=0}^{L-1} \epsilon_{\ell} \tilde{n}_{\ell}.
\end{equation*}
Here we have defined "rotated" creation operators as $\tilde{a}_{\ell}^{\dagger} = \sum_{j=0}^{L-1} u_{j\ell} a_j^{\dagger}$, and correspondingly for $\tilde{a}_{\ell}$. In other words, the column vectors in $u$ define a rotated orbital basis, and with respect to this basis, the Hamiltonian becomes a weighted sum of number operators. The rotated fermionic operators obey the same anti-commutation relations as the hitherto ones:
\begin{equation*}
\big\{\tilde{a}_i^{\dagger}, \tilde{a}_j^{\dagger}\big\} = 0, \quad \big\{\tilde{a}_i, \tilde{a}_j\big\} = 0, \quad \big\{\tilde{a}_i^{\dagger}, \tilde{a}_j\big\} = \delta_{i,j}
\end{equation*}
for all $i, j = 0, \dots L-1$, which can be verified by inserting their definition and using that $u$ is unitary.

In [1]:
import fermi_relations as fr
import numpy as np
import scipy.sparse.linalg as spla

In [2]:
# random number generator
rng = np.random.default_rng(45)

In [3]:
# number of modes (or orbitals) L
nmodes = 7

# random single-particle Hamiltonian
h = fr.crandn((nmodes, nmodes), rng)
h = 0.5*(h + h.conj().T)


In [4]:
# corresponding Hamiltonian on the whole Fock space
clist, alist, _ = fr.construct_fermionic_operators(nmodes)
hfock = sum(h[i, j] * (clist[i] @ alist[j])
            for i in range(nmodes)
            for j in range(nmodes))
# dimension scales as 2^L
hfock.shape

(128, 128)

In [5]:
# diagonalize 'h'
en, u = np.linalg.eigh(h)

# verify that 'u' indeed diagonalizes 'h'
print(np.linalg.norm(u @ np.diag(en) @ u.conj().T - h))
# 'u' should be unitary
print(np.linalg.norm(u.conj().T @ u - np.identity(nmodes)))

2.9844609677111104e-15
1.6389811762583227e-15


In [6]:
# represent H in terms of the number operators of the new basis
hfock_alt = sum(en[i] * fr.orbital_number_op(u[:, i]) for i in range(nmodes))
# compare (difference should be numerically zero)
spla.norm(hfock_alt - hfock)

np.float64(2.6727696998282556e-14)

Because of the representation of $H$, the Slater determinants constructed from the new orbitals are exactly the eigenvectors of $H$. Namely, let us select $N$ distinct columns from $u$, specified by the indices $0 \le i_0 < i_1 < \cdots < i_{N-1} < L$, and set $\Phi = \begin{pmatrix} u_{:, {i_0}} \vert \cdots \vert u_{:, {i_{N-1}}} \end{pmatrix} \in \mathbb{C}^{L \times N}$. Then the corresponding Slater determinant
\begin{equation*}
\ket{\Phi}_{\text{SD}} = a_{u_{:, {i_0}}}^\dagger \cdots a_{u_{:, {i_{N-1}}}}^\dagger \ket{\text{vac}} = \tilde{a}_{i_0}^\dagger \cdots \tilde{a}_{i_{N-1}}^\dagger \ket{\text{vac}}
\end{equation*}
is an eigenstate of $H$:
\begin{equation*}
H \ket{\Phi}_{\text{SD}} = \left(\sum_{n=0}^{N-1} \epsilon_{i_n}\right) \ket{\Phi}_{\text{SD}}.
\end{equation*}
Intuitively, if the $i$-th orbital is occupied, it contributes with $\epsilon_i$ to the overall energy eigenvalue. Since there are $2^L$ Slater determinants of this form ($2^L$ possibilities of which orbitals to select), we have found a complete orthonormal basis of eigenstates on the whole Fock space.

In [7]:
# construct such a Slater determinant (with three particles, as an example)
i0 = 1
i1 = 3
i2 = 6
phi = fr.slater_determinant(u[:, [i0, i1, i2]])

# verify that 'phi' is indeed an eigenstate of H
np.linalg.norm(hfock @ phi - (en[i0] + en[i1] + en[i2]) * phi)

np.float64(8.514751786723325e-16)